In [1]:
#export TRANSFORMERS_CACHE=/mnt/sda/dongkeun/huggingface

import os
os.environ['TRANSFORMERS_CACHE'] = '/mnt/sda/dongkeun/huggingface'

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM
import torch

DEVICE = 'cuda:2'

tokenizer = AutoTokenizer.from_pretrained('/home/dongkeun/axolotl/axolotl/checkpoints/orca-wiki-alpaca-lima-aya-elo')
model = AutoPeftModelForCausalLM.from_pretrained('/home/dongkeun/axolotl/axolotl/checkpoints/orca-wiki-alpaca-lima-aya-elo/checkpoint-141', torch_dtype=torch.bfloat16).to(DEVICE)

/home/dongkeun/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
content = '''
과일 가게에는 복숭아, 망고, 그리고 사과, 총 3개의 과일을 팔고 있습니다. 복숭아는 사과보다 비쌉니다. 망고가 가장 저렴합니다.
선택지:
(A) 복숭아가 두번째로 비쌉니다. (B) 망고가 두번째로 비쌉니다. (C) 사과가 두번째로 비쌉니다.

정답을 말하기 전, 생각을 단계별로 차근차근 먼저 말하고 마지막에 정답을 말하세요.
'''

messages = [
    {"role": "user", "content": content},
]

encodeds = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(encodeds)
encodeds = tokenizer.apply_chat_template(messages, return_tensors='pt', add_generation_prompt=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user

과일 가게에는 복숭아, 망고, 그리고 사과, 총 3개의 과일을 팔고 있습니다. 복숭아는 사과보다 비쌉니다. 망고가 가장 저렴합니다.
선택지:
(A) 복숭아가 두번째로 비쌉니다. (B) 망고가 두번째로 비쌉니다. (C) 사과가 두번째로 비쌉니다.

정답을 말하기 전, 생각을 단계별로 차근차근 먼저 말하고 마지막에 정답을 말하세요.
<|im_end|>
<|im_start|>assistant



In [5]:
model_inputs = encodeds.to(DEVICE)
generated_ids = model.generate(model_inputs, max_new_tokens=256, do_sample=False, eos_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<|im_start|>system
You are a helpful assistant.<|im_end|> 
<|im_start|>user

과일 가게에는 복숭아, 망고, 그리고 사과, 총 3개의 과일을 팔고 있습니다. 복숭아는 사과보다 비쌉니다. 망고가 가장 저렴합니다.
선택지:
(A) 복숭아가 두번째로 비쌉니다. (B) 망고가 두번째로 비쌉니다. (C) 사과가 두번째로 비쌉니다.

정답을 말하기 전, 생각을 단계별로 차근차근 먼저 말하고 마지막에 정답을 말하세요.
<|im_end|> 
<|im_start|>assistant
먼저 복숭아, 망고, 사과의 가격 순서를 파악해야 합니다. 복숭아는 사과보다 비싸다는 것을 알고 있으므로 복숭아 > 사과라고 쓸 수 있습니다. 또한 망고가 가장 저렴하다는 것을 알고 있으므로 망고 < 복숭아 < 사과라고 쓸 수 있습니다. 이제 망고가 두 번째로 저렴한 과일이라는 것을 알 수 있으므로 정답은 (B) 망고가 두 번째로 저렴하다입니다.<|im_end|>
